In [4]:
import os
from dotenv import load_dotenv
load_dotenv()

together_key = os.getenv("TOGETHER_API_KEY")

In [5]:
import dspy
lm = dspy.LM('mistralai/Mistral-7B-Instruct-v0.1', api_key=together_key, api_base='together_ai/togethercomputer/mistralai/Mistral-7B-Instruct-v0.1')
dspy.configure(lm=lm)

/home/francisca/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
## codigo dos direitos do autor
import json

with open('examples/direitos_autor_test.json', 'r') as file:
    data = json.load(file)

## LLM Metric

In [52]:

metricLM = dspy.LM('google/gemma-2-9b-it', api_key=together_key, api_base="together_ai/togethercomputer/")


In [ ]:
from datetime import datetime

class EvaluateAnswer(dspy.Signature):
    """Avalia a qualidade da resposta do sistema a uma certa questão baseado no criterio"""
    
    context = dspy.InputField(desc="O contexto necessário para responder à questão")
    criterion = dspy.InputField(desc="Critério de avaliação")
    question = dspy.InputField(desc="A pergunta perguntada ao sistema")
    model_answer = dspy.InputField(desc="A resposta do sistema que tens de avaliar")
    rating = dspy.OutputField(desc="Resultado entre 1 e 5. Apenas responde o número como um `int`, nada mais!")

class RAGMetricProgram(dspy.Module):
    def __init__(self):
        self.evaluator = dspy.ChainOfThought(EvaluateAnswer)
    
    def forward(self, context, question, predicted_answer, trace=None):
        init_time = datetime.now()
        faithful = "A resposta é suportada pelo contexto? Responde 'não' se inclui factos significantes não presentes no contexto"
        overall = f"Por favor avalia quão bem a resposta dada responde à questão baseado no contexto fornecido."

        with dspy.context(lm=metricLM):
            faithful = self.evaluator(context=context, criterion=faithful, question=question, model_answer=predicted_answer)
            overall = self.evaluator(context=context, criterion=overall, question=question, model_answer=predicted_answer)

        print(f"Faithful: {faithful.rating}")
        print(f"Overall: {overall.rating}")


        total = float(faithful.rating)*2 + float(overall.rating)
        final_time = datetime.now()

        print(f"Time passed: {final_time-init_time}\n")
        return total / 5.0

In [49]:
test_question = "Uma tradução é uma obra equiparada à obra originais?"
test_pred = "Sim, uma tradução é uma transformação de uma obra ainda que esta não seja objecto de protecção"

In [50]:
rag_metric = RAGMetricProgram()

In [ ]:
rag_metric(context="data", question=test_question,predicted_answer=test_pred)

## RAG

In [ ]:
import pydantic
from typing import List

class References(pydantic.BaseModel):
    article_title : str
    url : str
    date : str


class StructuredAnswer(pydantic.BaseModel):
    answer: str
    references: List[References]

In [ ]:
class GenerateAnswer(dspy.Signature):
    """Sendo tu um especialista da legislação portuguesa, responde à questão baseado EXCLUSIVAMENTE no contexto fornecido. Foca-te em fornecer informacões corretas e fundamenta com o contexto"""
    context = dspy.InputField(desc="Informação importante para responder à questão")
    question = dspy.InputField()
    answer : StructuredAnswer = dspy.OutputField(desc="Uma resposta detalhada e a lista de referencias utilizadas para responder à questão (título do artigo, url e data)")

In [ ]:
class RAG(dspy.Module):
    def __init__(self):
        super().__init__()
    
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
    
    def forward(self, question, context):
        pred = self.generate_answer(context=context, question=question).answer
        return dspy.Prediction(context=context, answer=pred, question=question)